In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 第一种方式，直接图像分块：
def image2emb_naive(image,patch_size,weight):
    #unfold 函数的返回值[N,C*W*H,L]
    patch = F.unfold(image,kernel_size=patch_size,stride=patch_size).transpose(-1,-2)
    #图像拆分成patch之后，和权重矩阵相乘，就得到了每个batch Embedding之后的维度：
    patch_embedding = torch.matmul(patch,weight)

    return patch_embedding

bs,ic,image_h,image_w = 1,3,8,8
patch_size = 4
model_dim = 8 #目标要输出的通道数目；
patch_depth = patch_size*patch_size*ic
image = torch.randn(bs,ic,image_h,image_w)
weight = torch.randn(patch_depth,model_dim)

patch_embedding_navie = image2emb_naive(image,patch_size,weight)
print(patch_embedding_navie.shape,patch_embedding_navie)

torch.Size([1, 4, 8]) tensor([[[ -1.3809,  -7.0244, -11.3916,   1.2869,  -4.8444, -10.3671,   5.6118,
           -5.1508],
         [ -4.9834,   0.0358, -15.2544,   3.6685,   8.6017,  -1.5891,   3.9160,
            6.3533],
         [-10.0968,   7.3256,  -0.7721,  -9.0098,   1.3375,  -0.2726,   3.7006,
            1.3974],
         [  8.0559,   5.9025,   0.8293,  -2.0891, -14.0389,   4.1494,  -2.2285,
           -4.6619]]])


In [6]:

# 第二种方式，卷积的方式：
def image2emb_conv(image,kernel,stride):
    conv_output = F.conv2d(image,kernel,stride=stride)
    bs,oc,ow,oh = conv_output.shape

    # 还是那个事，transfomer的输入形状是 NLC，L在图像中就是 W*H ，也就是序列长度；
    patch_embedding = conv_output.reshape((bs,oc,ow*oh)).transpose(-1,-2)
    return patch_embedding
bs,ic,image_h,image_w = 1,3,8,8
patch_size = 4
model_dim = 8 #目标要输出的通道数目；
patch_depth = patch_size*patch_size*ic
# image = torch.randn(bs,ic,image_h,image_w)
# weight = torch.randn(patch_depth,model_dim)

kernle = weight.transpose(0,1).reshape((-1,ic,patch_size,patch_size)) #kernel要满足 oc*ic*kh*kw
patch_embedding_conv = image2emb_conv(image,kernle,patch_size)
print(patch_embedding_conv.shape,patch_embedding_conv)

torch.Size([1, 4, 8]) tensor([[[ -1.3809,  -7.0244, -11.3916,   1.2869,  -4.8444, -10.3671,   5.6118,
           -5.1508],
         [ -4.9834,   0.0358, -15.2544,   3.6685,   8.6017,  -1.5891,   3.9160,
            6.3533],
         [-10.0968,   7.3256,  -0.7721,  -9.0098,   1.3375,  -0.2726,   3.7006,
            1.3974],
         [  8.0559,   5.9025,   0.8293,  -2.0891, -14.0389,   4.1494,  -2.2285,
           -4.6619]]])


In [ ]:
#-----------------------------------------------------------------------------------------
# 上面完成了图片的embedding